### Angry Birds Prototype -- Zach Quitkin and Evan Lyon's Final Project


In [1]:
#%conda install arcade
#%pip install arcade #alterate way to install

### Start Screen Code

In [2]:
import arcade
import glob

class InstructionView(arcade.View):
    
    def on_show_view(self):
        """ This is run once when we switch to this view """
        arcade.set_background_color(arcade.color.DARK_BLUE_GRAY)
        self.background = arcade.load_texture("AGB3.jpeg")
        #self.song_sound = arcade.load_sound()
        self.music = arcade.load_sound('AB1.wav')
        self.media_player = self.music.play()     
        # Reset the viewport, necessary if we have a scrolling game and we need
        # to reset the viewport back to the start so we can see what we draw.
        arcade.set_viewport(0, self.window.width, 0, self.window.height)
       # arcade.play_sound(self.song_sound)
        
    def on_draw(self):
        """ Draw this view """
        self.clear()
        arcade.draw_lrwh_rectangle_textured(0, 0,
                                            SCREEN_WIDTH, SCREEN_HEIGHT,
                                            self.background)
        arcade.draw_text("Angry Birds Prototype", self.window.width / 2, self.window.height-80,
                         arcade.color.BLACK, font_size=80, anchor_x="center", font_name="Kenney Pixel")
        arcade.draw_text("Click to advance", self.window.width / 2, self.window.height-120,
                         arcade.color.BLACK, font_size=30, anchor_x="center",font_name="Kenney Pixel")
        
        
        
    def on_mouse_press(self, _x, _y, _button, _modifiers):
        """ If the user presses the mouse button, start the game. """
        self.media_player.pause()
        game_view = GameView()
        game_view.setup()
        self.window.show_view(game_view)
        
        

### Player Class

In [3]:
import arcade
import glob


class PlayerCharacter(arcade.Sprite): #player class has sprite, hitbox, etc
    def __init__(self):
        super().__init__()
        
        # Used for flipping between image sequences
        self.cur_texture = 0
        self.walk_textures = []
        self.scale = 1
        
        # Images from Kenney.nl's Asset Pack 3
        
        main_path = "Angrybirds*.png"
        for filename in sorted(glob.glob(main_path)):
            self.walk_textures += [arcade.load_texture(filename)]

        # Set the initial texture
        self.texture = self.walk_textures[0]

        # Hit box will be set based on the first image used. If you want to specify
        # a different hit box, you can do it like the code below.
        # set_hit_box = [[-22, -64], [22, -64], [22, 28], [-22, 28]]
        self.hit_box = self.texture.hit_box_points

    def update_animation(self, delta_time: float = 1 / 60):
        # Walking animation, code to sense if the PlayerCharacter is moving
        if self.change_x > 0 or self.change_x < 0: 
            print('\rx=%s,y=%s'%(int(self.center_x),int(self.center_y)))
        self.texture = self.walk_textures[self.cur_texture]

### Main Game -- Level Class

In [4]:
# Constantscoin1.wav
import arcade
SCREEN_WIDTH = 1400
SCREEN_HEIGHT = 800
SCREEN_TITLE = "Angry Birds Prototype"

# Constants used to scale our sprites from original size = 1
TILE_SCALING = 0.5
COIN_SCALING = 0.5

# Movement speed of player, in pixels per frame
PLAYER_MOVEMENT_SPEED = 3
GRAVITY = 1
PLAYER_JUMP_SPEED = 3

class GameView(arcade.View): #load sounds from disk to RAM in constructor
    def __init__(self):
        super().__init__() #window
        
        self.tile_map, self.scene, self.player_sprite = None, None, None
        self.physics_engine, self.camera, self.gui_camera = None, None, None
        self.score = 0
        # Load sounds
        self.collect_coin_sound = arcade.load_sound("coin1.wav")
        self.jump_sound = arcade.load_sound("bird_sound.wav")
        arcade.set_background_color(arcade.csscolor.CORNFLOWER_BLUE)
        
        
        self.num_x = 0
        self.num_y = 0
        
        #arcade.draw_text(("X velocity =", self.xvel), self.window.width / 4, self.window.height-80,
                     #    arcade.color.BLACK, font_size=60, anchor_x="center", font_name="Kenney Future")
        #arcade.draw_text(("Y velocity =,", self.yvel), self.window.width / 4, self.window.height-120,
                     #    arcade.color.BLACK, font_size=60, anchor_x="center", )
        

    def setup(self): #starts the game (or restarts)
        # Keep track of the score
        self.score = 0
        self.num_x = 0
        self.num_y = 0
        
        # Set up the Cameras used to scroll the platforms, etc and Scene
        self.camera = arcade.Camera(SCREEN_WIDTH, SCREEN_HEIGHT)
        self.gui_camera = arcade.Camera(SCREEN_WIDTH, SCREEN_HEIGHT)
        self.scene = arcade.Scene() # Scene has everything you need in one structure
        
        self.player_sprite = PlayerCharacter()
        self.player_sprite.center_x = 128
        self.player_sprite.center_y = 228
        self.scene.add_sprite("Player", self.player_sprite)
        
        # This shows using a loop to place multiple ground sprites horizontally
        for x in range(0, 2500, 64):
            wall = arcade.Sprite("grassMid.png", TILE_SCALING)
            wall.center_x = x
            wall.center_y = 32
            self.scene.add_sprite("Platforms", wall)
            
        # Put some crates on the ground [x_pos,y_pos]
        coordinate_list = [[512, 96], [256, 96], [768, 96]] #these are [x,y coordinates]
        for coordinate in coordinate_list:
            # Add a crate on the ground
            wall = arcade.Sprite( "boxCrate_double.png", TILE_SCALING)
            wall.position = coordinate           #set the wall position
            self.scene.add_sprite("Platforms", wall) #add it to the scene
            
        wall = arcade.Sprite( "C2atapult.png")
        wall.position = [128, 130]           #set the wall position
        self.scene.add_sprite("Platforms", wall)  #add it to the scene
        
        for x in range(200, 2500, 64):
            wall = arcade.Sprite("invisible.png")
            wall.center_x = 2000
            wall.center_y = x
            #wall.position = [150, 140]           #set the wall position
            self.scene.add_sprite("Platforms", wall)  #add it to the scene
            
            
        for x in range(625, 1250, 625):
            coin = arcade.Sprite("pig2.png", COIN_SCALING)
            coin.center_x = x
            coin.center_y = 110
            self.scene.add_sprite("Coins", coin)

        # Create the 'physics engine'
        self.physics_engine = arcade.PhysicsEnginePlatformer(
            self.player_sprite,
            gravity_constant=GRAVITY, 
            walls=self.scene["Platforms"]
        )

    def on_draw(self):
        """Render the screen."""
        # Clear the screen to the background color
        self.clear()
        # Activate the game camera
        self.camera.use()
        # Draw our Scene
        self.scene.draw()
        # Activate the GUI camera before drawing GUI elements
        self.gui_camera.use()

        # Draw our score on the screen, scrolling it with the viewport
        score_text = f"Score: {self.score}"
        arcade.draw_text(score_text,10,10,arcade.csscolor.WHITE,18)
        
        arcade.draw_text("Use UP/DOWN to increase/decrease X Velocity" 
                         " and LEFT/RIGHT to increase/decrease y velocity. SPACE to fire. Click"
                         " to restart",10,600,arcade.csscolor.BLACK,18)
        #Display velocity:
        x_text = f"X Velocity: {self.num_x}"
        arcade.draw_text(x_text,10,550,arcade.csscolor.BLACK,14)
        y_text = f"Y Velocity: {self.num_y}"
        arcade.draw_text(y_text,10,500,arcade.csscolor.BLACK,14)
        


    def on_key_press(self, key, modifiers):
        """Called whenever a key is pressed."""
        if key == arcade.key.SPACE:
            if self.physics_engine.can_jump():
                self.player_sprite.change_y = PLAYER_JUMP_SPEED * self.num_y
                self.player_sprite.change_x = PLAYER_MOVEMENT_SPEED * self.num_x
                arcade.play_sound(self.jump_sound)
        #Change Velocity:
        elif key == arcade.key.UP:  
            self.num_x = self.num_x+1
        elif key == arcade.key.DOWN:
            self.num_x = self.num_x-1
        elif key == arcade.key.LEFT:
            self.num_y = self.num_y-1
        elif key == arcade.key.RIGHT:
            self.num_y = self.num_y+1
            
    def on_mouse_press(self, _x, _y, _button, _modifiers):
        """ If the user presses the mouse button, start the game. """
        game_view = GameView()
        game_view.setup()
        self.window.show_view(game_view)
        
    def on_key_release(self, key, modifiers):
        """Called when the user releases a key."""
        if key == arcade.key.LEFT:
            self.player_sprite.change_x = 0
        elif key == arcade.key.RIGHT:
            self.player_sprite.change_x = 0

    def center_camera_to_player(self):
        screen_center_x = self.player_sprite.center_x - (self.camera.viewport_width / 2)
        screen_center_y = self.player_sprite.center_y - (self.camera.viewport_height / 2)
        if screen_center_x < 0: screen_center_x = 0
        if screen_center_y < 0: screen_center_y = 0
        player_centered = screen_center_x, screen_center_y
        self.camera.move_to(player_centered)

    def on_update(self, delta_time):
        """Movement and game logic"""
        # Move the player with the physics engine
        self.physics_engine.update()
        
        # Update Animations
        self.scene.update_animation(delta_time, ["Player"])
        
        # See if we hit any coins
        coin_hit_list = arcade.check_for_collision_with_list(
            self.player_sprite, self.scene["Coins"]
        )

        # Loop through each coin we hit (if any) and remove it
        for coin in coin_hit_list:
            # Remove the coin
            coin.remove_from_sprite_lists()
            # Play a sound
            arcade.play_sound(self.collect_coin_sound)
            # Add one to the score
            self.score += 1
            
        if self.score == 1:
            view = GameOverView()
            self.window.show_view(view)

        # Position the camera
        self.center_camera_to_player()

### You Win Screen

In [5]:
class GameOverView(arcade.View):
    """ View to show when game is over """

    def on_show_view(self):
        """ This is run once when we switch to this view """
        #self.background = arcade.set_background_color(arcade.color.DARK_BLUE_GRAY)
        self.background = arcade.load_texture("bluebackgrond.png")
        self.music = arcade.load_sound('AB1.wav')
        self.media_player = self.music.play()     
        # Reset the viewport, necessary if we have a scrolling game and we need
        # to reset the viewport back to the start so we can see what we draw.
        arcade.set_viewport(0, self.window.width, 0, self.window.height)
        
        
    def on_draw(self):
        """ Draw this view """
        self.clear()
        arcade.draw_lrwh_rectangle_textured(0, 0,
                                            SCREEN_WIDTH, SCREEN_HEIGHT,
                                            self.background)
        arcade.draw_text("You Win!", self.window.width / 2, self.window.height/2,
                         arcade.color.WHITE, font_size=100, anchor_x="center", font_name="Kenney Pixel")
        arcade.draw_text("Click to restart", self.window.width / 2, (self.window.height/2)-40,
                         arcade.color.WHITE, font_size=30, anchor_x="center",font_name="Kenney Pixel")
        
    def on_mouse_press(self, _x, _y, _button, _modifiers):
        """ If the user presses the mouse button, re-start the game. """
        self.media_player.pause()
        game_view = GameView()
        game_view.setup()
        self.window.show_view(game_view)

### Makes a class that runs the game

In [6]:
def main():
    
    window = arcade.Window(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
    start_view = InstructionView()
    window.show_view(start_view)
    arcade.run()

In [ ]:
main()

2023-12-08 11:34:52.123 python[64141:2277123] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9b5ad725c0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-12-08 11:34:52.124 python[64141:2277123] Warning: Expected min height of view: (<NSButton: 0x7f9b5b26e560>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-12-08 11:34:52.126 python[64141:2277123] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9b56d1ab70>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.
2023-12-08 11:34:52.127 python[64141:2277123] Warning: Expected min height of view: (<NSPopoverTouchBarItemButton: 0x7f9b5b27dba0>) to be less than or equal to 30 but got a height of 32.000000. This error will be logged once per view in violation.


x=146,y=257
x=164,y=264
x=182,y=270
x=200,y=275
x=218,y=279
x=236,y=282
x=254,y=284
x=272,y=285
x=290,y=285
x=308,y=284
x=326,y=282
x=344,y=279
x=362,y=275
x=380,y=270
x=398,y=264
x=416,y=257
x=434,y=249
x=452,y=240
x=470,y=230
x=488,y=219
x=506,y=207
x=524,y=194
x=542,y=180
x=560,y=172
x=578,y=171
x=596,y=169
x=614,y=166
x=632,y=162
